In [27]:
import tifffile
import zarr

In [15]:
z = zarr.open('/mnt/raid_nvme/inference_out/predictions.zarr', mode='r')
print(z.tree())

norms = z['normals_final']
sheet = z['sheet_final']

s_n = norms[:,1000, :, ]
s_s = sheet[1000]

tifffile.imwrite('s_n.tif', s_n)
tifffile.imwrite('s_s.tif', s_s)

/
 ├── normals_count (2000, 1500, 1750) float32
 ├── normals_final (3, 2000, 1500, 1750) uint16
 ├── normals_sum (3, 2000, 1500, 1750) float32
 ├── sheet_count (2000, 1500, 1750) float32
 ├── sheet_final (2000, 1500, 1750) uint8
 └── sheet_sum (2000, 1500, 1750) float32


In [31]:
import zarr
import tifffile
import os

# Open the Zarr array
raw = zarr.open("/home/sean/Desktop/s1_segments/3336/predictions.zarr", mode='r')
ink = raw['ink_final']

# Get the shape of the array
z_layers, height, width = ink.shape  # Assuming shape is (Z, Y, X)

# Create output directory if it doesn't exist
output_dir = "/home/sean/Desktop/s1_segments/3336/ink_layers"
os.makedirs(output_dir, exist_ok=True)

# Iterate through Z layers
for z in range(z_layers):
    # Read the z-layer
    layer = ink[z, :, :]

    # Create output filename
    output_path = os.path.join(output_dir, f"{z:04d}.tif")

    # Save as TIFF
    tifffile.imwrite(output_path, layer)
    print(f"Saved layer {z} to {output_path}")

Saved layer 0 to /home/sean/Desktop/s1_segments/3336/ink_layers/0000.tif
Saved layer 1 to /home/sean/Desktop/s1_segments/3336/ink_layers/0001.tif
Saved layer 2 to /home/sean/Desktop/s1_segments/3336/ink_layers/0002.tif
Saved layer 3 to /home/sean/Desktop/s1_segments/3336/ink_layers/0003.tif
Saved layer 4 to /home/sean/Desktop/s1_segments/3336/ink_layers/0004.tif
Saved layer 5 to /home/sean/Desktop/s1_segments/3336/ink_layers/0005.tif
Saved layer 6 to /home/sean/Desktop/s1_segments/3336/ink_layers/0006.tif
Saved layer 7 to /home/sean/Desktop/s1_segments/3336/ink_layers/0007.tif
Saved layer 8 to /home/sean/Desktop/s1_segments/3336/ink_layers/0008.tif
Saved layer 9 to /home/sean/Desktop/s1_segments/3336/ink_layers/0009.tif
Saved layer 10 to /home/sean/Desktop/s1_segments/3336/ink_layers/0010.tif
Saved layer 11 to /home/sean/Desktop/s1_segments/3336/ink_layers/0011.tif
Saved layer 12 to /home/sean/Desktop/s1_segments/3336/ink_layers/0012.tif
Saved layer 13 to /home/sean/Desktop/s1_segments

In [4]:
import shutil
import os

def copy_and_number(source_image, start_number, num_copies):
    # Check if source file exists
    if not os.path.exists(source_image):
        print(f"Error: Source file '{source_image}' not found")
        return

    # Get the directory and file extension
    directory = os.path.dirname(source_image) or '.'
    _, ext = os.path.splitext(source_image)

    # Create numbered copies
    for i in range(num_copies):
        new_number = start_number + i
        new_filename = f"{new_number}{ext}"
        new_path = os.path.join(directory, new_filename)

        try:
            shutil.copy2(source_image, new_path)
            print(f"Created: {new_filename}")
        except Exception as e:
            print(f"Error creating {new_filename}: {e}")

# Example usage
if __name__ == "__main__":
    source_image = "/home/sean/Desktop/s1_segments/5753/20230702185753_inklabels.png"
    start_number = 1
    num_copies =20

    copy_and_number(source_image, start_number, num_copies)

Created: 1.png
Created: 2.png
Created: 3.png
Created: 4.png
Created: 5.png
Created: 6.png
Created: 7.png
Created: 8.png
Created: 9.png
Created: 10.png
Created: 11.png
Created: 12.png
Created: 13.png
Created: 14.png
Created: 15.png
Created: 16.png
Created: 17.png
Created: 18.png
Created: 19.png
Created: 20.png


In [1]:
import os
import glob
import re
from tqdm import tqdm

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'([0-9]+)', s)]

def check_layers_in_range(parent_folder, start=25, stop=35):
    """
    Go through each segment folder in `parent_folder` (ones that have a 'layers' subdirectory)
    and check if the slice indices [start..stop] are valid based on the number of layer files.

    If any slice index is out of range, we print a warning message.
    """

    # 1) Find all subdirectories that contain a 'layers' folder
    segment_folders = [
        f.path for f in os.scandir(parent_folder)
        if f.is_dir() and os.path.exists(os.path.join(f.path, 'layers'))
    ]

    if not segment_folders:
        raise ValueError(f"No valid segment folders found in {parent_folder}")

    print(f"Found {len(segment_folders)} segment folders to check.")

    # 2) For each segment, gather layer files and sort them the same way
    for folder in tqdm(segment_folders, desc="Checking segments"):
        seg_name = os.path.basename(folder)
        layers_dir = os.path.join(folder, 'layers')

        # Collect and sort .tif layer files
        layers = glob.glob(os.path.join(layers_dir, '*.tif'))
        layers.sort(key=natural_sort_key)

        if not layers:
            print(f"WARNING: No .tif files found in {layers_dir}")
            continue

        # 3) Determine how many slices we plan to read
        num_slices = stop - start + 1

        # 4) Check if each requested slice index is within range
        #    of the sorted `layers` list
        out_of_range = False
        for idx in range(num_slices):
            file_index = start + idx
            if file_index >= len(layers):
                # This means we'd "fall back" to the last layer file in your original script
                print(
                    f"Out-of-range: Segment '{seg_name}' "
                    f"needs slice index {file_index} but only has {len(layers)} files."
                )
                out_of_range = True
                # We don't break here because we may want to see *all* out-of-range indices

        # 5) Optional: If you only want to print a single message for each segment
        # that has any missing slices, you could do:
        if out_of_range:
            print(f" -> Segment '{seg_name}' does NOT have all slices for [{start}..{stop}].")
        else:
            print(f" -> Segment '{seg_name}' is good (has at least {stop+1} files).")

if __name__ == "__main__":
    parent_folder = "/home/sean/Desktop/s1_segments/"
    check_layers_in_range(parent_folder, start=25, stop=35)


Found 8 segment folders to check.


Checking segments: 100%|██████████| 8/8 [00:00<00:00, 2137.36it/s]

Out-of-range: Segment '4423' needs slice index 25 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 26 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 27 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 28 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 29 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 30 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 31 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 32 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 33 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 34 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 35 but only has 21 files.
 -> Segment '4423' does NOT have all slices for [25..35].
Out-of-range: Segment '0901' needs slice index 25 but only has 21 files.
Out-of-range: Segment '0901' needs slice index 26 but only has 21 

In [1]:
import os
import sys
import argparse

import numpy as np
import zarr
import cv2

def slice_normals_zarr(zarr_path, dataset_name, output_dir):
    """
    Reads a 3-channel dataset from a zarr file and writes each z-slice as a 3-channel JPG image.

    - The dataset is expected to be shaped (3, Z, Y, X), i.e. (channels, depth, height, width).
    - The script will transpose it to (height, width, channels) for OpenCV.
    - Assumes final data is either already uint8 or uint16. If it is uint16, it applies a simple
      min/max scaling to convert into 8-bit for JPG writing.
    """

    print(f"Opening Zarr store: {zarr_path}")
    zarr_store = zarr.open(zarr_path, mode='r')

    if dataset_name not in zarr_store:
        raise ValueError(f"Dataset '{dataset_name}' not found in '{zarr_path}'")

    dataset = zarr_store[dataset_name]
    if dataset.ndim != 4:
        raise ValueError(
            f"Expected dataset '{dataset_name}' to have 4 dimensions (3, Z, Y, X). "
            f"Got shape {dataset.shape} instead."
        )

    c, Z, Y, X = dataset.shape
    if c != 3:
        raise ValueError(f"Expected 3 channels for dataset '{dataset_name}'. Got shape={dataset.shape}")

    print(f"Found dataset '{dataset_name}' with shape={dataset.shape}")
    print(f"Writing slices to: {output_dir}")
    os.makedirs(output_dir, exist_ok=True)

    # Read metadata about the data type.
    dtype = str(dataset.dtype)
    print(f"Dataset dtype={dtype}")

    for z in range(Z):
        # Read one z-slice: shape = (3, Y, X)
        slice_3d = dataset[:, z, :, :]

        # Transpose to (Y, X, 3) for OpenCV
        slice_3d = np.transpose(slice_3d, (1, 2, 0))

        # If dtype is uint16, do min/max scaling to 8 bits for visualization
        if 'uint16' in dtype:
            # Simple stretch from [0..65535] -> [0..255]
            slice_3d_8u = cv2.convertScaleAbs(slice_3d, alpha=(255.0/65535.0))
        else:
            # Assume data is already in the correct 0..255 range
            slice_3d_8u = slice_3d.astype(np.uint8, copy=False)

        # OpenCV expects BGR order. If your data is already BGR, you can keep it as is.
        # If your data is in (R, G, B), you can swap it here:
        # slice_3d_8u = slice_3d_8u[..., ::-1]  # (R, G, B) -> (B, G, R)

        # Construct the output path
        out_path = os.path.join(output_dir, f"slice_{z:04d}.jpg")

        cv2.imwrite(out_path, slice_3d_8u)

        if z % 50 == 0:
            print(f"Wrote z-slice {z} -> {out_path}")

    print("Done writing slices.")


def main():


    slice_normals_zarr(
        zarr_path="/mnt/raid_nvme/inference_out/predictions.zarr",
        dataset_name="normals_final",
        output_dir="/mnt/raid_nvme/inference_out/normals_final_slices"
    )


if __name__ == '__main__':
    main()


Opening Zarr store: /mnt/raid_nvme/inference_out/predictions.zarr
Found dataset 'normals_final' with shape=(3, 2000, 1500, 1750)
Writing slices to: /mnt/raid_nvme/inference_out/normals_final_slices
Dataset dtype=uint16
Wrote z-slice 0 -> /mnt/raid_nvme/inference_out/normals_final_slices/slice_0000.jpg
Wrote z-slice 50 -> /mnt/raid_nvme/inference_out/normals_final_slices/slice_0050.jpg
Wrote z-slice 100 -> /mnt/raid_nvme/inference_out/normals_final_slices/slice_0100.jpg
Wrote z-slice 150 -> /mnt/raid_nvme/inference_out/normals_final_slices/slice_0150.jpg
Wrote z-slice 200 -> /mnt/raid_nvme/inference_out/normals_final_slices/slice_0200.jpg
Wrote z-slice 250 -> /mnt/raid_nvme/inference_out/normals_final_slices/slice_0250.jpg


KeyboardInterrupt: 